In [ ]:
##########################################################################################################################################
############# This script extracts MODIS mean annual net photosynthesis values at each of the soil algae data point position #############
############################################ This script was used to produce tha data in Fig.1 ##########################################
##########################################################################################################################################

In [28]:
import ee # Import the Python API package
ee.Authenticate() # Authenticate to the Earth Engine servers
ee.Initialize() # Initialize the API

Enter verification code:  4/1AX4XfWg_PmVA5qYUCi12RM4a6bJhKl5bZy5yjW8AACGo1023awWrdmRunJ8



Successfully saved authorization token.


In [29]:
import geemap # The geemap Python package is built upon the ipyleaflet and folium packages and implements several methods for interacting with Earth Engine data layers

In [30]:
Map = geemap.Map(center=[0,0], zoom=2)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Toggl…

In [31]:
# Loading sampling points stored in personnal repository
ProductivityPts = ee.FeatureCollection('users/romainwalcker/Phototrophs_Productivity_AllPts')
print('Size of the productivity data collection',ProductivityPts.size().getInfo())

Size of the productivity data collection 102


In [32]:
# Loading MODIS annual net photosynthesis data (in kgC/m^2/yr scaled at 0.0001) here converted in gC/m^2/yr
MOD17A3HGF = ee.ImageCollection('MODIS/006/MOD17A3HGF')
nppMean = MOD17A3HGF.select('Npp').mean().divide(10000).multiply(1000).rename('nppMean') # Long term mean net photosynthesis
nppMedian = MOD17A3HGF.select('Npp').median().divide(10000).multiply(1000).rename('nppMedian') # Long term median
nppMin = MOD17A3HGF.select('Npp').min().divide(10000).multiply(1000).rename('nppMin') # Long term min
nppMax = MOD17A3HGF.select('Npp').max().divide(10000).multiply(1000).rename('nppMax') # Long term max

NPPStack = nppMean.addBands(nppMedian).addBands(nppMin).addBands(nppMax) # stacked into a multiband image

In [46]:
NPP_Sampled_AllPts = NPPStack.sampleRegions(collection=ProductivityPts, geometries= True, scale=500)

In [51]:
# Map mean MODIS annual net photosynthesis data
vis = {'palette': ['330044', '220066', '1133cc', '33dd00', 'ffda21', 'ff6622', 'd10000'],'min':0,'max':1500,'opacity': 0.5}
Map.addLayer(nppMean, vis,'nppMean')

In [50]:
# Map sampling points
Map.addLayer(ProductivityPts, {},'Sampling points')

In [48]:
# Export an ee.FeatureCollection as an Earth Engine asset.
taskParams = {'driveFolder': 'Jupyter','fileFormat': 'CSV'}
task = ee.batch.Export.table.toDrive(NPP_Sampled_AllPts,str('NPP_Sampled_AllPts'),**taskParams)
task.start()